In [75]:
def parse_iterator(source_name):
    lines = []
    with open(source_name) as file:
        lines = file.readlines()
    
    #eliminate new line char
    for idx, line in enumerate(lines):
        if line[-1] == '\n':
            lines[idx] = line[0:-1]
        
    num_test_cases = int(lines[0])
    maze_size_idx = 1
    for test_id in range(num_test_cases):
        # parse maze shape
        maze_size = [int(i) for i in lines[maze_size_idx].split()]
        row_count, col_count = maze_size                
        
        # init maze, exit, start positions
        maze = [[0] * col_count for i in range(row_count)]
        exit = None
        start = None
        
        # parse maze
        row = maze_size_idx + 1
        for row_idx in range(row_count):
            line = lines[row]
            for col_idx in range(col_count):
                if line[col_idx] == 'E':
                    exit = (row_idx, col_idx)
                if line[col_idx] == 'Y':
                    start = (row_idx, col_idx)
                if line[col_idx] == 'X':
                    maze[row_idx][col_idx] = 1
            row += 1
        yield(maze, start, exit)
        maze_size_idx = row #set next maze size position
    

In [104]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


def astar(maze, start, end):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = 0
    start_node.h = 0
    start_node.f = 0
    end_node = Node(None, end)
    end_node.g = 0
    end_node.h = 0
    end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(-1, 0), (1, 0),(0, -1), (0, 1)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure no one is seated
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1 # since unweighted graph
            # Manhatan distane since mazae is 2D grid
            child.h = (abs(child.position[0] - end_node.position[0])) + (abs(child.position[1] - end_node.position[1]))
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)

In [105]:
def convert_to_actions(path):
    if path == None:
        return None
    action_map = {(0,-1):'L',(0,1): 'R', (-1, 0): 'N', (1,0): 'S'}
    new_path = []
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i+1]        
        delta = (end[0] - start[0], end[1] - start[1])
        new_path.append(action_map[delta])
    return new_path

In [106]:
def find_maximal_repeating_ending(action_path):
    if action_path == None:
        return None
    path_len = len(action_path)
    current_max_repeat = 0
    current_max_sub_string = ""
    for pattern_size in range(1, int(path_len/2) + 1):

        max_possible_repeats =  int(path_len / pattern_size)
        pattern = action_path[path_len-pattern_size:]
        num_repeats = 0
        for sub_i in range(max_possible_repeats):
            start = path_len - (sub_i + 1) * pattern_size
            end = path_len - sub_i * pattern_size
            sub = action_path[start:end]
            if(sub == pattern):
                num_repeats = num_repeats + 1
            else:
                break
        if num_repeats > current_max_repeat:
            current_max_sub_string = pattern
            current_max_repeat = num_repeats
    return current_max_sub_string,current_max_repeat

In [107]:
def convert_step_format(action_path):
    if action_path == None:
        return None
    pattern, repeats = find_maximal_repeating_ending(action_path)
    if repeats <= 1:
        return action_path
    new_path = action_path[:len(action_path) - repeats * len(pattern) + 1]
    new_path.append(f'M({len(new_path)})')
    return new_path

In [108]:
for maze, start, exit in parse_iterator('small-testdata.in'):
    path = astar(maze, start, exit)
    action_path = convert_to_actions(path)
    new_path = convert_step_format(action_path)
    res = "IMPOSSIBLE" if new_path == None else str(len(new_path)) 
    print(new_path, res)

['L', 'N', 'L', 'M(3)'] 4
['N', 'L', 'L', 'S', 'L', 'L', 'N'] 7
['L', 'L', 'N', 'M(3)'] 4
['L', 'L', 'N', 'N', 'L', 'N', 'N', 'R'] 8
None IMPOSSIBLE


In [43]:
pattern_size

2